Added function
1. Detect multiple video files
2. Print the number of face from each video file

In [1]:
import cv2
import mediapipe as mp
import time

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# def apply_face_mosaic(image, bbox):
#     x, y, w, h = bbox
#     roi = image[y:y+h, x:x+w]
#     roi = cv2.resize(roi, (10, 10), interpolation=cv2.INTER_NEAREST)
#     roi = cv2.resize(roi, (w, h), interpolation=cv2.INTER_NEAREST)
#     image[y:y+h, x:x+w] = roi
#     return image

video_files = ['audience.mp4', 'street.mp4', 'conversation.mp4']  # List of video file paths

# Iterate over the video files
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)

    # Initialize the Mediapipe face detection module
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7) as face_detection:
        num_detection = 0  # Initialize the counter for the number of detections
        prev_faces = [] # List to store the previously detected face coordinates
        
        start_time = time.time() # Start measuring the execution time

        while cap.isOpened() and num_detection < 30:
            success, frame = cap.read()
            if not success:
                break

            target_frame = frame.copy()  # copy frame

            # Perform face detection using Mediapipe on the current frame
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)

            # Draw face detection annotations on the frame and crop card region
            if results.detections:
                for detection in results.detections:
                    mp_drawing.draw_detection(frame, detection)

                    # Get the bounding box coordinates of the detected face
                    bbox = detection.location_data.relative_bounding_box
                    xmin, ymin, width, height = int(bbox.xmin * frame.shape[1]), int(bbox.ymin * frame.shape[0]), \
                                               int(bbox.width * frame.shape[1]), int(bbox.height * frame.shape[0])

                    # Check if the current face overlaps with any previously detected faces
                    is_duplicate = False
                    for prev_face in prev_faces:
                        if abs(xmin - prev_face[0]) < prev_face[2] and abs(ymin - prev_face[1]) < prev_face[3]:
                            is_duplicate = True
                            break

                    if not is_duplicate:
                        num_detection += 1
                        consecutive_duplicates = 0
                        while consecutive_duplicates < 2000:
#                         num_detection += 1
#                         consecutive_duplicates = 0

                            # Crop the face region based on the face bounding box
                            face_crop = frame[ymin:ymin + height, xmin:xmin + width]

                            # Apply face mosaic to the cropped face region
#                             face_crop = apply_face_mosaic(face_crop, (0, 0, width, height))

                            cv2.imshow(f'face_crop_{num_detection}', face_crop)
                            cv2.imwrite(f'face_crop_{num_detection}.png', face_crop)

                            # Add the current face coordinates to the list of previously detected faces
                            prev_faces.append((xmin, ymin, width, height))
                        
                            consecutive_duplicates += 1
                            if consecutive_duplicates >= 2000:
                                break

            cv2.imshow('frame', frame)
            # cv2.imshow('contour', target_frame)

            if cv2.waitKey(1) == ord('q'):
                break
                                
    end_time = time.time()  # Stop measuring the execution time

cap.release()
cv2.destroyAllWindows()
        
print(f"Number of faces detected in {video_file}: {num_detection}")

execution_time = end_time - start_time
print("Total execution time:", execution_time, "seconds")


Number of faces detected in conversation.mp4: 3
Total execution time: 26.122710466384888 seconds
